In [1]:
#載入資料
import pandas as pd  
import numpy as np  
df = pd.read_csv('breast-cancer-wisconsin.csv')

In [2]:
#fill ?
df.replace('?',np.nan,inplace = True)
df['Bare_Nuclei'] = pd.to_numeric(df['Bare_Nuclei'])
for i in df.columns:
    df[i].fillna(value=df.groupby('Class')[i].transform('mean'), inplace=True)

In [3]:
from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier

In [4]:
#split data
X = df.iloc[:,1:10]
y = df.iloc[:,10].values

In [5]:
#Accuracy
def Accuracy(y_pred,y_test):
    count = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test[i]:
            count = count + 1
    return count / len(y_pred)

In [6]:
#pca
from numpy.linalg import eig
def to_pca(X_train, X_test):
    k0 = 1
    x_mean = X_train.mean().values
    C = X_train - x_mean
    C_covariance = C.cov().values
    eigenvalues, eigenvectors = eig(C_covariance)
    for k in range(len(eigenvalues)):
        if sum(eigenvalues[:k+1])/sum(eigenvalues) <= 0.9:
            k0 = k0 + 1
    C_test = X_test - x_mean
    return C.dot(eigenvectors[:,:k0]),C_test.dot(eigenvectors[:,:k0])

In [7]:
#5-nn手寫算法
def knn_5(X_train,X_test,y_train):
    pre = []
    for i in X_test.values:
        dist = []
        for j in range(len(X_train)):
            dist.append((np.linalg.norm(i-X_train.values[j]),y_train[j]))
        sort_dist = sorted(dist, key=lambda x:x[0])
        vote = []
        for i in sort_dist[:5]:
            vote.append(i[1])
        pre.append(max(vote,key=vote.count))
    return pre

In [9]:
#train
Accuracy_list_ori = []
Accuracy_list_pca = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)#random split
    #no PCA
# 下3行套件測試用，因手寫算法執行時間有些久
#    KNNclassifier = KNeighborsClassifier(n_neighbors=5,p = 2)#p=2 Euclidean
#    KNNclassifier.fit(X_train, y_train)
#    y_pred = KNNclassifier.predict(X_test)
    #手寫算法
    y_pred = knn_5(X_train,X_test,y_train)
    Accuracy_list_ori.append(Accuracy(y_pred,y_test))

    #PCA
    pca_X_train,pca_X_test = to_pca(X_train, X_test)
# 下3行套件測試用，因手寫算法執行時間有些久
#    KNNclassifier_pca = KNeighborsClassifier(n_neighbors=5,p = 2)#p=2 Euclidean
#    KNNclassifier_pca.fit(pca_X_train, y_train)
#    pca_y_pred = KNNclassifier_pca.predict(pca_X_test)
    #手寫算法
    pca_y_pred = knn_5(pca_X_train,pca_X_test,y_train)
    Accuracy_list_pca.append(Accuracy(pca_y_pred,y_test))
print ('Each accuracy no PCA:', Accuracy_list_ori)
print ('Each accuracy with PCA:', Accuracy_list_pca)
print ('Average accuracy no PCA:', sum(Accuracy_list_ori) / 10)
print ('Average accuracy with PCA:', sum(Accuracy_list_pca) / 10)
#差不多準度

Each accuracy no PCA: [0.9857142857142858, 0.9714285714285714, 0.9666666666666667, 0.9619047619047619, 0.9619047619047619, 0.9714285714285714, 0.9619047619047619, 0.9904761904761905, 0.9666666666666667, 0.9523809523809523]
Each accuracy with PCA: [0.9714285714285714, 0.9809523809523809, 0.9809523809523809, 0.9619047619047619, 0.9571428571428572, 0.9666666666666667, 0.9666666666666667, 0.9904761904761905, 0.9714285714285714, 0.9619047619047619]
Average accuracy no PCA: 0.9690476190476192
Average accuracy with PCA: 0.9709523809523809
